<a href="https://colab.research.google.com/github/pko89403/Recsys_test/blob/master/implicit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pwd

/content


In [2]:
!nvidia-smi

Sat Feb 15 01:40:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
!nvidia-smi --query | fgrep 'Product Name'

    Product Name                    : Tesla T4


In [0]:
!mkdir data

In [0]:
!wget -q http://www.grouplens.org/system/files/ml-1m.zip ./data

In [6]:
!unzip -o ml-1m -d data

Archive:  ml-1m.zip
   creating: data/ml-1m/
  inflating: data/ml-1m/movies.dat   
  inflating: data/ml-1m/ratings.dat  
  inflating: data/ml-1m/README       
  inflating: data/ml-1m/users.dat    


In [0]:
cp -rf ./data ./drive/My\ Drive/

In [7]:
!pip install implicit==v0.4.0

     |████████████████████████████████| 1.1MB 32.0MB/s 
  Created wheel for implicit: filename=implicit-0.4.0-cp36-cp36m-linux_x86_64.whl size=3440522 sha256=4e3b4dc980e80842322a505f570cbb756eafe8d76f000480dc974a9eb99062fe
  Stored in directory: /root/.cache/pip/wheels/fd/d1/66/84e83191d9f626c521271023e4ef83e634e3b8c154f2c4b2da
Successfully built implicit


In [14]:
!pip freeze | grep implicit

implicit==0.4.0


In [0]:
import implicit

In [21]:
!du -sh ./data/ml-1m/*

168K	./data/ml-1m/movies.dat
24M	./data/ml-1m/ratings.dat
8.0K	./data/ml-1m/README
132K	./data/ml-1m/users.dat


In [5]:
!head -n 5 ./data/ml-1m/ratings.dat

head: cannot open './data/ml-1m/ratings.dat' for reading: No such file or directory


In [0]:
import pandas as pd
from scipy.sparse import csr_matrix # compressed sparse row
from scipy.io import mmwrite
import numpy as np

In [0]:
ratings_path = "/content/drive/My Drive/data/ml-1m/ratings.dat"
movies_path = "/content/drive/My Drive/data/ml-1m/movies.dat"

ratings= pd.read_csv(ratings_path, header=None, sep='::', engine='python')
movies = pd.read_csv(movies_path, header=None, sep='::', engine='python')

In [13]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [14]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
0    1000209 non-null int64
1    1000209 non-null int64
2    1000209 non-null int64
3    1000209 non-null int64
dtypes: int64(4)
memory usage: 30.5 MB


In [16]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
0    3883 non-null int64
1    3883 non-null object
2    3883 non-null object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [18]:
ratings.columns = ["uid", "iid", "ratings", "timestamp"]
ratings.head()

,uid,iid,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [19]:
movies.columns = ["iid", "movie_name", "genre"]
movies.head()

,iid,movie_name,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [21]:
ratings.describe() # Ratings Mean is 3.58

,uid,iid,ratings,timestamp
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,9.722437e+08
std,1.728413e+03,1.096041e+03,1.117102e+00,1.215256e+07
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08
25%,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08
50%,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08
75%,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08
max,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09


In [0]:
def parse_moviename(movie_name):
  return movie_name.replace(' ', '_').encode('utf-8').decode('ascii', 'ignore')


In [0]:
iid_to_movie_name = dict( zip(movies.iid.tolist(), movies.movie_name.tolist()) )

In [0]:
iid_to_movie_name = {iid:parse_moviename(movie_name) for (iid, movie_name) in iid_to_movie_name.items()}

In [59]:
iid_to_movie_name[1]

'Toy_Story_(1995)'

In [0]:
uid_to_idx = { uid:idx for (idx, uid) in enumerate(ratings.uid.unique().tolist()) } # 평점을 매긴 유니크한 uid

In [0]:
iid_to_idx = {iid: idx for (idx, iid) in enumerate(ratings.iid.unique().tolist())} # 평점이 매겨진 유니크한 iid

In [0]:
idx_to_movie_name = {idx : iid_to_movie_name[iid] for (iid, idx) in iid_to_idx.items()} # idx -> movie_name

In [79]:
len(ratings) / ( len(uid_to_idx) * len(iid_to_idx) ) # sparsity calc

0.044683625622312845

In [0]:
row = ratings.iid.tolist()
col = ratings.uid.tolist()
dat = ratings.ratings.tolist()

In [0]:
row = [iid_to_idx[r] for r in row]
col = [uid_to_idx[c] for c in col]

In [0]:
train_matrix = csr_matrix((dat, (row, col)), shape=(1+np.max(row), 1+np.max(col)))

In [91]:
train_matrix.shape

(3706, 6040)